In [2]:
import pandas as pd
import numpy as np
import os

In [5]:
print('開始する都道府県コードを入力してください')
start_pref = input()
print(f'開始する都道府県コード : {start_pref}')

print('終了する都道府県コードを入力してください')
finish_pref = input()
print(f'終了する都道府県コード : {finish_pref}')


#浸水深スケーリング
def flood_rank_scaling(column):
    flood_rank = np.where((column == 11) | (column == 21), 1, 
                    np.where((column == 12) | (column == 22), 2, 
                        np.where((column == 13) | (column == 23), 3,
                            np.where((column == 14) | (column == 24) | (column == 25) | (column == 26), 4, 5))))
    return flood_rank
     


#最大―最小スケーリング
def min_max_scaling_numpy(column):
    #指定のカラムをnumpy配列に
    array = column.to_numpy()
    
    #最大―最小スケーリングを実施
    min_val = np.min(array)
    max_val = np.max(array)
    scaled_array = (array - min_val) / (max_val - min_val)
    return scaled_array






def subsidy(burden_df):
    #補助金とそれに対応する移転希望者の割合の辞書を作成
    subsidy_dic = {0.1:0.05,
                   0.2:0.07,
                   0.3:0.08,
                   0.4:0.1,
                   0.5:0.12,
                   0.6:0.14,
                   0.7:0.17,
                   0.8:0.18,
                   0.9:0.19,
                   1.0:0.21}

    # 総和を保存するための空のデータフレームを作成
    subsidy_df = pd.DataFrame(columns=['subsidy_ratio', 'transfer_ratio', 'total_subsidy', 'measures_damage_cost','measures_total_cost', 'without_measures_cost'])

    #補助金毎にループ
    for col, frac in subsidy_dic.items():
        #burden_dfにdamage_priceに対する補助する金額の列を追加
        burden_df[col] = burden_df['house_price']*col


        # データフレームから指定された割合で行をランダムに選ぶ
        sample_df = burden_df[['house_price', 'damage_price', col]].sample(frac=frac)

        # 選択された行に含まれる数値の総和を計算
        total_sum = sample_df[col].sum()
        
        
        #選択された住宅の移転費用と損害額を比較する必要がある．いらない変数といる変数を分ける．
        
        #補助金と損害額の総和
        
        #施策なしの損害額
        without_measures_cost = burden_df['damage_price'].sum()
        #施策有の損害額
        measures_damage_cost = without_measures_cost - sample_df['damage_price'].sum()
        #補助金と損害額の総和
        measures_total_cost = measures_damage_cost + total_sum
        

        # 総和を一時的なデータフレームに保存
        temp_df = pd.DataFrame({
            'subsidy_ratio': [f"subsidy_{col * 100}%"],
            'transfer_ratio': [f"transfer_{frac * 100}%"],
            'total_subsidy': [total_sum],
            'measures_damage_cost':[measures_damage_cost],
            'measures_total_cost':[measures_total_cost],
            'without_measures_cost':[without_measures_cost]
            })

        # 結果をメインのデータフレームに追加
        subsidy_df = pd.concat([subsidy_df, temp_df], ignore_index=True)
        

    return subsidy_df

    










%cd "G:/マイドライブ/akiyamalab/卒論/負担率総集編"
n = int(start_pref)
while n <= int(finish_pref):
    burden_path = f'./dev/建物家財損害額/{n}_lost.csv'
    burden_df = pd.read_csv(burden_path, encoding='shift-jis')
    
    #浸水ランクが5段階と7段階があるので5段かにスケーリングする
    burden_df['flood_rank'] = flood_rank_scaling(burden_df['A31_001'])
    
    #最大―最小スケーリング
    burden_df['estimated_age_scaled'] = min_max_scaling_numpy(burden_df['estimated_age'])
    burden_df['flood_rank_scaled'] = min_max_scaling_numpy(burden_df['flood_rank'])
    
    
    # subsidy(burden_df)
    
    n += 1

開始する都道府県コードを入力してください
開始する都道府県コード : 33
終了する都道府県コードを入力してください
終了する都道府県コード : 33
G:\マイドライブ\akiyamalab\卒論\負担率総集編


In [50]:
subsidy(burden_df)

,subsidy_ratio,transfer_ratio,total_subsidy,measures_damage_cost,measures_total_cost,without_measures_cost
0,subsidy_30.0%,transfer_64.5%,1.701710e+12,8.050577e+11,2.506768e+12,2.273164e+12
1,subsidy_50.0%,transfer_64.60000000000001%,2.842290e+12,8.045300e+11,3.646820e+12,2.273164e+12
2,subsidy_70.0%,transfer_64.8%,3.989081e+12,8.009014e+11,4.789982e+12,2.273164e+12
3,subsidy_100.0%,transfer_66.8%,5.878401e+12,7.550209e+11,6.633422e+12,2.273164e+12


In [6]:
burden_df

,tatemon_id,x,y,geometry,floor,AREA,atype2,total_area,KEY_CODE,estimated_structure,...,house_damage_price,assets,children,adult,add_assets,assets_damage_price,total_lost,flood_rank,estimated_age_scaled,flood_rank_scaled
0,1573,133.797213,34.672247,POINT (133.797212782 34.6722473045),0,166.805,1364,166.805,331012270,1,...,4.452973e+06,11500000,2,2,4200000,9420000.0,1.387297e+07,2,0.500000,0.25
1,1688,133.797788,34.673460,POINT (133.79778766 34.6734596316),0,184.710,1364,184.710,331012270,1,...,9.415541e+06,11500000,2,2,4200000,6280000.0,1.569554e+07,1,0.333333,0.00
2,1694,133.797064,34.672907,POINT (133.797064016 34.6729072982),0,102.940,1364,102.940,331012270,1,...,8.833405e+06,11500000,2,4,5800000,6920000.0,1.575340e+07,1,0.000000,0.00
3,1701,133.797125,34.672804,POINT (133.797124865 34.6728042894),0,68.600,1364,68.600,331012270,1,...,3.496866e+06,3000000,0,5,4000000,2800000.0,6.296866e+06,1,0.333333,0.00
4,1703,133.796789,34.672734,POINT (133.79678866 34.6727343175),0,115.070,1364,115.070,331012270,1,...,9.874295e+06,3000000,0,3,2400000,2160000.0,1.203429e+07,1,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231777,19890,133.816148,34.860425,POINT (133.816147573 34.8604254275),0,107.045,1364,107.045,33681059005,1,...,5.456589e+06,5000000,4,2,6800000,4720000.0,1.017659e+07,1,0.333333,0.00
231778,19898,133.817725,34.860710,POINT (133.817724701 34.8607098307),0,118.770,1364,118.770,33681059006,1,...,1.019180e+07,11500000,2,2,4200000,6280000.0,1.647180e+07,1,0.000000,0.00
231779,19908,133.816064,34.861105,POINT (133.816063964 34.861104545),0,168.965,1364,168.965,33681059006,1,...,1.281733e+07,11500000,2,2,4200000,6280000.0,1.909733e+07,1,0.166667,0.00
231780,19909,133.816188,34.860946,POINT (133.816187744 34.8609455599),0,181.605,1364,181.605,33681059006,1,...,9.257264e+06,11500000,2,2,4200000,6280000.0,1.553726e+07,1,0.333333,0.00


In [3]:
%cd "G:/マイドライブ/akiyamalab/卒論/負担率総集編"
burden_path = f'./dev/損害額/33_step8_1_output.csv'
burden_df = pd.read_csv(burden_path, encoding='shift-jis')

G:\マイドライブ\akiyamalab\卒論\負担率総集編


In [5]:
import numpy as np

# 重みの計算（damage_priceが大きいほど重みが小さくなるように設定）
weights = 1 / (burden_df['damage_price'] + 1)  # 0除算を避けるために1を加える
weights /= weights.sum()  # 重みの合計が1になるように正規化

# データの10%を重みに基づいてランダムに抽出
sample_size = int(len(burden_df) * 0.1)  # 抽出する行数（全体の10%）
sampled_data = burden_df.sample(n=sample_size, weights=weights, random_state=1)

sampled_data.head()  # 抽出されたデータの最初の数行を表示

,tatemon_id,x,y,floor,AREA,atype2,total_area,KEY_CODE,estimated_structure,estimated_age,A31_001,cost,dpc_rate,age,dmg_rate,house_price,damage_price
120457,27045,133.948296,34.683880,0,63.220,1364,63.220,331020490,1,7,13,256000,0.036,50.0,0.75,1.618432e+07,6.069120e+05
223911,50700,133.696008,34.629778,0,72.540,1364,72.540,33202209057,1,7,15,256000,0.036,50.0,0.80,1.857024e+07,7.428096e+05
74,9127,133.813706,34.679553,0,132.145,1364,132.145,331012280,1,7,14,256000,0.036,50.0,0.80,3.382912e+07,1.353165e+06
91707,57669,133.908388,34.626896,0,60.885,1364,60.885,331040062,1,4,12,256000,0.036,25.5,0.60,1.558656e+07,1.625366e+06
44246,137710,133.923982,34.666961,0,29.575,1364,29.575,33101040001,1,6,12,256000,0.036,45.5,0.60,7.571200e+06,2.271360e+05


In [7]:
sampled_data

,tatemon_id,x,y,floor,AREA,atype2,total_area,KEY_CODE,estimated_structure,estimated_age,A31_001,cost,dpc_rate,age,dmg_rate,house_price,damage_price
120457,27045,133.948296,34.683880,0,63.220,1364,63.220,331020490,1,7,13,256000,0.036,50.0,0.75,1.618432e+07,6.069120e+05
223911,50700,133.696008,34.629778,0,72.540,1364,72.540,33202209057,1,7,15,256000,0.036,50.0,0.80,1.857024e+07,7.428096e+05
74,9127,133.813706,34.679553,0,132.145,1364,132.145,331012280,1,7,14,256000,0.036,50.0,0.80,3.382912e+07,1.353165e+06
91707,57669,133.908388,34.626896,0,60.885,1364,60.885,331040062,1,4,12,256000,0.036,25.5,0.60,1.558656e+07,1.625366e+06
44246,137710,133.923982,34.666961,0,29.575,1364,29.575,33101040001,1,6,12,256000,0.036,45.5,0.60,7.571200e+06,2.271360e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14110,46030,133.870486,34.645637,0,55.510,1364,55.510,331011370,1,7,12,256000,0.036,50.0,0.60,1.421056e+07,4.263168e+05
137673,66883,133.982000,34.647463,0,70.390,1364,70.390,331020240,1,5,13,256000,0.036,35.5,0.75,1.801984e+07,6.757440e+05
144018,50344,133.962812,34.684315,0,70.495,1364,70.495,33102053104,1,5,12,256000,0.036,35.5,0.60,1.804672e+07,5.414016e+05
51634,117066,133.916795,34.640650,0,96.665,1364,96.665,331011300,1,5,11,256000,0.036,35.5,0.40,2.474624e+07,4.949248e+05


In [6]:
weights

0         1.085253e-06
1         3.226271e-07
2         6.556284e-07
3         5.132579e-07
4         5.470825e-07
              ...     
333097    2.480271e-06
333098    8.856450e-07
333099    4.741658e-07
333100    8.936224e-07
333101    3.446658e-07
Name: damage_price, Length: 333102, dtype: float64